In [2]:
import pandas as pd

In [5]:
df = pd.read_excel("shl_tests.xlsx")

In [6]:
df.head()

,Assessment Name,Description,URL,Remote Testing,Duration,Test Type
0,Adobe Experience Manager (New),Multi-choice test that measures the knowledge ...,https://www.shl.com/solutions/products/product...,Yes,17 minutes,K
1,Adobe Photoshop CC,The Adobe Photoshop CC test measures knowledge...,https://www.shl.com/solutions/products/product...,Yes,20 minutes,K
2,Aeronautical Engineering (New),Multi-choice test that measures the knowledge ...,https://www.shl.com/solutions/products/product...,Yes,10 minutes,K
3,Aerospace Engineering (New),Multi-choice test that measures the conceptual...,https://www.shl.com/solutions/products/product...,Yes,10 minutes,K
4,Agile Software Development,Multi-choice test that measures the knowledge ...,https://www.shl.com/solutions/products/product...,Yes,Not found,K


In [7]:
df = df.dropna(subset=['Assessment Name', 'Description'])
df = df.fillna('N/A')
data = df.to_dict(orient='records')

## Step 2: Embed the Descriptions Using a Text Embedding Model

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
descriptions = [item['Description'] for item in data]
embeddings = model.encode(descriptions, show_progress_bar=True)

C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python 

In [9]:
embeddings

array([[-0.00088001, -0.03840558, -0.00837101, ..., -0.03682839,
        -0.05852675, -0.00376767],
       [-0.01923871,  0.00951021,  0.01012801, ...,  0.01862345,
        -0.00336232,  0.02133397],
       [ 0.08047848, -0.05689639, -0.01991648, ..., -0.01301624,
        -0.09166525,  0.03044293],
       ...,
       [ 0.00998266,  0.02559538, -0.02432171, ...,  0.00880277,
         0.01266969, -0.01123319],
       [-0.07703921,  0.02433646,  0.01084779, ...,  0.01548038,
        -0.0081068 , -0.03208848],
       [ 0.02948253, -0.04839161, -0.04197101, ..., -0.00515717,
        -0.01163711, -0.02531304]], dtype=float32)

## Step 3: Build a Semantic Search Function

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_top_n(query, n=10):
    query_embedding = model.encode([query])
    sims = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(sims)[::-1][:n]
    return [data[i] for i in top_indices]

## Step 4: Build a REST API

In [12]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


In [13]:
from fastapi import FastAPI, Request
from pydantic import BaseModel

app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/recommend")
def recommend(query: Query):
    results = get_top_n(query.text)
    return results

## Step 5: Build a Web UI for Demo

In [14]:
import streamlit as st

st.title("SHL Assessment Recommender")
user_input = st.text_area("Enter job description or role")

if st.button("Get Recommendations"):
    results = get_top_n(user_input)
    st.table(pd.DataFrame(results))

2025-04-05 14:24:34.652 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-05 14:24:35.317 
  command:

    streamlit run C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-05 14:24:35.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-05 14:24:35.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-05 14:24:35.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-05 14:24:35.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-05 14:24:35.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-05 14:24:35.